In [1]:
import json
import re

import numpy as np
import pandas as pd
from unidecode import unidecode

pd.options.mode.chained_assignment = None

/usr/local/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df_elections = pd.read_csv('/mnt/data/asset_evolution_elections_2018.csv', dtype={'voter_id': np.str})
df_elections

,id,name,voter_id,num_of_elections_run,year,state,party,position,result,sum_of_assets_declared,num_of_assets_declared
0,530217,ALFREDO CARDOSO COSTA,000481911325,7,2004,PA,PT,VEREADOR(A),ELEITO,NaN,0
1,530217,ALFREDO CARDOSO COSTA,000481911325,7,2006,PA,PT,DEPUTADO(A) ESTADUAL,SUPLENTE,251333.67,10
2,530217,ALFREDO CARDOSO COSTA,000481911325,7,2008,PA,PT,VEREADOR(A),ELEITO,371218.40,7
3,530217,ALFREDO CARDOSO COSTA,000481911325,7,2010,PA,PT,DEPUTADO(A) ESTADUAL,SUPLENTE,519814.79,14
4,530217,ALFREDO CARDOSO COSTA,000481911325,7,2012,PA,PT,PREFEITO(A),NÃO ELEITO,739094.99,11
5,530217,ALFREDO CARDOSO COSTA,000481911325,7,2014,PA,PT,DEPUTADO(A) ESTADUAL,SUPLENTE,1049678.89,12
6,530217,ALFREDO CARDOSO COSTA,000481911325,7,2018,PA,PT,DEPUTADO(A) ESTADUAL,0,814371.24,11
7,1179243,DIRCEU TEN CATEN PIES,055759191309,2,2014,PA,PT,DEPUTADO(A) ESTADUAL,ELEITO POR QP,136148.00,3
8,1179243,DIRCEU TEN CATEN PIES,055759191309,2,2018,PA,PT,DEPUTADO(A) ESTADUAL,0,376456.25,9
9,2235,ELIANA FERREIRA EMERICK,001380992003,3,1998,DF,PMDB,DEPUTADO(A) FEDERAL,SUPLENTE,NaN,0


In [3]:
# Filtra por últimas eleições
df_elections = df_elections[df_elections.year.isin([2010, 2014, 2018])]

In [4]:
# Selectiona candidatos que concorreram ao menos em 2 eleições
counts = df_elections.name.value_counts()
candidates = counts[counts >= 2].index

In [5]:
# Separa o dataframe apenas usando candidatos de ao menos 2 eleições
df = df_elections[df_elections.name.isin(candidates)]
df.head()

,id,name,voter_id,num_of_elections_run,year,state,party,position,result,sum_of_assets_declared,num_of_assets_declared
3,530217,ALFREDO CARDOSO COSTA,000481911325,7,2010,PA,PT,DEPUTADO(A) ESTADUAL,SUPLENTE,519814.79,14
5,530217,ALFREDO CARDOSO COSTA,000481911325,7,2014,PA,PT,DEPUTADO(A) ESTADUAL,SUPLENTE,1049678.89,12
6,530217,ALFREDO CARDOSO COSTA,000481911325,7,2018,PA,PT,DEPUTADO(A) ESTADUAL,0,814371.24,11
7,1179243,DIRCEU TEN CATEN PIES,055759191309,2,2014,PA,PT,DEPUTADO(A) ESTADUAL,ELEITO POR QP,136148.00,3
8,1179243,DIRCEU TEN CATEN PIES,055759191309,2,2018,PA,PT,DEPUTADO(A) ESTADUAL,0,376456.25,9


In [6]:
df.shape

(6190, 11)

In [7]:
# Substitui valores ausentes por 0
df.sum_of_assets_declared.fillna(0, inplace=True)

In [8]:
# Transformando em milhoes para facilitar
df.sum_of_assets_declared = df.sum_of_assets_declared / 10**6

In [9]:
df.head()

,id,name,voter_id,num_of_elections_run,year,state,party,position,result,sum_of_assets_declared,num_of_assets_declared
3,530217,ALFREDO CARDOSO COSTA,000481911325,7,2010,PA,PT,DEPUTADO(A) ESTADUAL,SUPLENTE,0.519815,14
5,530217,ALFREDO CARDOSO COSTA,000481911325,7,2014,PA,PT,DEPUTADO(A) ESTADUAL,SUPLENTE,1.049679,12
6,530217,ALFREDO CARDOSO COSTA,000481911325,7,2018,PA,PT,DEPUTADO(A) ESTADUAL,0,0.814371,11
7,1179243,DIRCEU TEN CATEN PIES,055759191309,2,2014,PA,PT,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.136148,3
8,1179243,DIRCEU TEN CATEN PIES,055759191309,2,2018,PA,PT,DEPUTADO(A) ESTADUAL,0,0.376456,9


# Análise por pessoa

In [10]:
df.position = pd.Categorical(df.position,
                             categories=['PRESIDENTE(A) DA REPÚBLICA',  'GOVERNADOR(A)', 'SENADOR(A)', 
                                         'DEPUTADO(A) FEDERAL', 'DEPUTADO(A) ESTADUAL',
                                         'VICE-GOVERNADOR(A)', '2º SUPLENTE', 
                                         '1º SUPLENTE SENADOR(A)', '1º SUPLENTE', '2º SUPLENTE SENADOR(A)', 
                                         'VICE-PRESIDENTE(A)', 'DEPUTADO(A) DISTRITAL'][::-1],
                             ordered=True)

In [11]:
df.result = pd.Categorical(df.result, 
                           categories=['ELEITO', 'ELEITO POR QP', 'NÃO ELEITO', 
                                       'REJEITADO', 'SUPLENTE', 'SUBSTITUÍDO', 
                                       '2o TURNO','0'][::-1],
                           ordered=True)

In [12]:
df_result = df.groupby(['name', 'year'])['result'].max().unstack().reset_index()
df_result.rename(columns={
    2010.0: '2010-resultado',
    2014.0: '2014-resultado',
    2018.0: '2018-resultado',
}, inplace=True)

del df_result.columns.name
df_result.set_index('name', inplace=True)
df_result.reset_index()

df_result.drop(['2018-resultado'], axis=True, inplace=True)
df_result.head()

,2010-resultado,2014-resultado
name,,
ABDALA HABIB FRAXE JUNIOR,ELEITO,ELEITO POR QP
ABEILARD GOULART DE SOUZA FILHO,NaN,SUPLENTE
ABEL DO PORTO SANTOS,NÃO ELEITO,NaN
ABEL MOACIR BURGDURFF DE MORAES,NaN,NÃO ELEITO
ABIDENE SALUSTIANO DA SILVA,NaN,SUPLENTE


In [13]:
df_position = df.groupby(['name', 'year'])['position'].max().unstack().reset_index()
df_position.rename(columns={
    2010.0: '2010-cargo',
    2014.0: '2014-cargo',
    2018.0: '2018-cargo',
}, inplace=True)

del df_position.columns.name
df_position.set_index('name', inplace=True)
df_position.reset_index()
df_position.head()

,2010-cargo,2014-cargo,2018-cargo
name,,,
ABDALA HABIB FRAXE JUNIOR,DEPUTADO(A) ESTADUAL,DEPUTADO(A) ESTADUAL,DEPUTADO(A) ESTADUAL
ABEILARD GOULART DE SOUZA FILHO,NaN,DEPUTADO(A) ESTADUAL,DEPUTADO(A) FEDERAL
ABEL DO PORTO SANTOS,DEPUTADO(A) FEDERAL,NaN,DEPUTADO(A) FEDERAL
ABEL MOACIR BURGDURFF DE MORAES,NaN,1º SUPLENTE,DEPUTADO(A) ESTADUAL
ABIDENE SALUSTIANO DA SILVA,NaN,DEPUTADO(A) FEDERAL,DEPUTADO(A) ESTADUAL


In [14]:
df_assets = df.groupby(['name', 'year'])['sum_of_assets_declared'].max().unstack().reset_index()
df_assets.rename(columns={
    2010.0: '2010-soma-bens',
    2014.0: '2014-soma-bens',
    2018.0: '2018-soma-bens',
}, inplace=True)


del df_assets.columns.name
df_assets.set_index('name', inplace=True)
df_assets.reset_index()
df_assets.head()

,2010-soma-bens,2014-soma-bens,2018-soma-bens
name,,,
ABDALA HABIB FRAXE JUNIOR,0.550,0.550000,0.550000
ABEILARD GOULART DE SOUZA FILHO,NaN,0.112152,0.152298
ABEL DO PORTO SANTOS,0.035,NaN,0.025000
ABEL MOACIR BURGDURFF DE MORAES,NaN,0.005000,0.303000
ABIDENE SALUSTIANO DA SILVA,NaN,0.028000,0.135000


In [15]:
dd = pd.concat([df_result, df_position, df_assets], axis=1)

dd['2010-2014'] = dd['2014-soma-bens'] - dd['2010-soma-bens']
dd['2010-2014-percent'] = ((dd['2014-soma-bens'] - dd['2010-soma-bens']) / dd['2010-soma-bens'])*100
dd['2014-2018'] = dd['2018-soma-bens'] - dd['2014-soma-bens']
dd['2014-2018-percent'] = ((dd['2018-soma-bens'] - dd['2014-soma-bens']) / dd['2014-soma-bens'])*100
dd['2010-2018'] = dd['2018-soma-bens'] - dd['2010-soma-bens']
dd['2010-2018-percent'] = ((dd['2018-soma-bens'] - dd['2010-soma-bens']) / dd['2010-soma-bens'])*100


#dd['2010-2014-percent-pretty'] = dd['2010-2014-percent'].apply(lambda x: '{:.1f}%'.format(x))
dd['2014-2018-percent-pretty'] = dd['2014-2018-percent'].apply(lambda x: '{:.1f}%'.format(x))
dd['2010-2018-percent-pretty'] = dd['2010-2018-percent'].apply(lambda x: '{:.1f}%'.format(x))

df_final = dd[[
    #'2010-cargo', '2010-resultado', '2010-soma-bens', 
    '2014-cargo', '2014-resultado', '2014-soma-bens', #'2010-2014', '2010-2014-percent', '2010-2014-percent-pretty',
    '2018-cargo', '2018-soma-bens', '2014-2018', '2014-2018-percent', '2014-2018-percent-pretty', 
    #'2010-2018', '2010-2018-percent', '2010-2018-percent-pretty',
]]

In [16]:
drop = [
    #'2010-2014-percent', 
    '2014-2018-percent', 
    #'2010-2018-percent'
]

In [17]:
top30_2018 = df_final.sort_values(by='2018-soma-bens', ascending=False).head(30)
top30_2018.drop(drop, axis=1, inplace=True)
top30_2018.to_csv('/mnt/data/top-30-ricos-nas-eleicoes-2018.csv')
top30_2018.head(10)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent-pretty
name,,,,,,,
OTAVIANO OLAVO PIVETTA,NaN,NaN,NaN,VICE-GOVERNADOR(A),379.456427,NaN,nan%
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.619703,DEPUTADO(A) ESTADUAL,153.719389,153.099686,24705.3%
PEDRO CHAVES DOS SANTOS FILHO,NaN,NaN,NaN,SENADOR(A),130.494561,NaN,nan%
MAURO MENDES FERREIRA,NaN,NaN,NaN,GOVERNADOR(A),113.453807,NaN,nan%
IVONE MEZZOMO CASSOL,SENADOR(A),NÃO ELEITO,56.276546,1º SUPLENTE,84.714269,28.437724,50.5%
NILSO JOSE BERLANDA,DEPUTADO(A) ESTADUAL,SUPLENTE,6.266501,DEPUTADO(A) ESTADUAL,64.809477,58.542976,934.2%
REINALDO AZAMBUJA SILVA,GOVERNADOR(A),ELEITO,27.440416,GOVERNADOR(A),38.698697,11.258282,41.0%
DAMINA DE CARVALHO PEREIRA,DEPUTADO(A) FEDERAL,ELEITO POR QP,38.827718,DEPUTADO(A) FEDERAL,37.586761,-1.240957,-3.2%
EDENILSO ROSSI ARNALDI,DEPUTADO(A) FEDERAL,0,27.077399,DEPUTADO(A) FEDERAL,33.589670,6.512271,24.1%


In [18]:
top30_crescimento_absoluto_2018 = df_final.sort_values(by='2014-2018', ascending=False).head(30)
top30_crescimento_absoluto_2018.drop(drop, axis=1, inplace=True)
top30_crescimento_absoluto_2018.to_csv(
    '/mnt/data/top-30-que-enriqueceram-desde-as-eleicoes-2014-valor-absoluto.csv')
top30_crescimento_absoluto_2018.head(10)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent-pretty
name,,,,,,,
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.619703,DEPUTADO(A) ESTADUAL,153.719389,153.099686,24705.3%
NILSO JOSE BERLANDA,DEPUTADO(A) ESTADUAL,SUPLENTE,6.266501,DEPUTADO(A) ESTADUAL,64.809477,58.542976,934.2%
IVONE MEZZOMO CASSOL,SENADOR(A),NÃO ELEITO,56.276546,1º SUPLENTE,84.714269,28.437724,50.5%
DILMAR DAL BOSCO,DEPUTADO(A) ESTADUAL,ELEITO POR QP,2.394865,DEPUTADO(A) ESTADUAL,22.252168,19.857304,829.2%
MARINALDO ROSENDO DE ALBUQUERQUE,DEPUTADO(A) FEDERAL,ELEITO POR QP,13.026175,DEPUTADO(A) FEDERAL,32.235112,19.208937,147.5%
MIGUEL MOUBADDA HADDAD,DEPUTADO(A) FEDERAL,ELEITO POR QP,11.383318,DEPUTADO(A) FEDERAL,29.316575,17.933257,157.5%
ALFREDO COTAIT NETO,2º SUPLENTE,NÃO ELEITO,4.746242,1º SUPLENTE,16.050417,11.304175,238.2%
REINALDO AZAMBUJA SILVA,GOVERNADOR(A),ELEITO,27.440416,GOVERNADOR(A),38.698697,11.258282,41.0%
ANTONIO JORGE GODINHO,DEPUTADO(A) ESTADUAL,SUPLENTE,1.649338,SENADOR(A),12.261525,10.612187,643.4%


In [19]:
df_sem_percent_infinito = df_final[df_final['2014-2018-percent']!= np.inf]
top30_crescimento_percent_2018 = df_sem_percent_infinito.sort_values(by='2014-2018-percent', ascending=False).head(30)
top30_crescimento_percent_2018.drop(drop, axis=1, inplace=True)
top30_crescimento_percent_2018.to_csv(
    '/mnt/data/top-30-que-enriqueceram-desde-as-eleicoes-2014-valor-percent.csv')

top30_crescimento_percent_2018.head(10)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent-pretty
name,,,,,,,
RUBENS SANCHEZ PROENCA,DEPUTADO(A) FEDERAL,0,3.000000e-08,DEPUTADO(A) FEDERAL,0.012000,0.012000,39999900.0%
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,6.197028e-01,DEPUTADO(A) ESTADUAL,153.719389,153.099686,24705.3%
CARLOS DO SOCORRO GUERREIRO VAZ,DEPUTADO(A) FEDERAL,SUPLENTE,3.000000e-03,DEPUTADO(A) FEDERAL,0.507000,0.504000,16800.0%
CLEINALDO CASTRO LOPES,DEPUTADO(A) ESTADUAL,SUPLENTE,3.500000e-03,DEPUTADO(A) ESTADUAL,0.369485,0.365985,10456.7%
NAIARA LOURENCO,DEPUTADO(A) DISTRITAL,NÃO ELEITO,2.000000e-02,DEPUTADO(A) FEDERAL,2.046000,2.026000,10130.0%
GUSTAVO DE BIASE NUNES DA ROCHA,DEPUTADO(A) FEDERAL,SUPLENTE,2.500000e-03,DEPUTADO(A) FEDERAL,0.171000,0.168500,6740.0%
BESALIEL DE OLIVEIRA RODRIGUES,DEPUTADO(A) ESTADUAL,NÃO ELEITO,1.300000e-02,DEPUTADO(A) ESTADUAL,0.880000,0.867000,6669.2%
ABEL MOACIR BURGDURFF DE MORAES,1º SUPLENTE,NÃO ELEITO,5.000000e-03,DEPUTADO(A) ESTADUAL,0.303000,0.298000,5960.0%
SILVIO LUIZ RODRIGUES RAMALHO,DEPUTADO(A) ESTADUAL,SUPLENTE,5.000000e-03,DEPUTADO(A) ESTADUAL,0.244000,0.239000,4780.0%


# Análise de Evolução Patrimonial por Partido

In [20]:
df.party = pd.Categorical(df.party,
                             categories=['PDT', 'PSD', 'PSTU', 'PV', 'PSDB', 'PMDB', 'PROS', 'PSOL', 'PP',
                                         'PRP', 'PSB', 'PT', 'PSC', 'PRB', 'PMN', 'PR', 'PPS', 'PTC',
                                         'PODE', 'PSL', 'PRTB', 'PTN', 'PSDC', 'DEM', 'PMB', 'PHS', 'SD',
                                         'REDE', 'PPL', 'PTB', 'PEN', 'PCB', 'NOVO', 'PCO'],
                             ordered=True)

In [21]:
df_evolution = dd[[
    '2010-2014', '2010-2014-percent',
    '2014-2018', '2014-2018-percent',
]]
df_evolution.head()

,2010-2014,2010-2014-percent,2014-2018,2014-2018-percent
name,,,,
ABDALA HABIB FRAXE JUNIOR,0.0,0.0,0.000000,0.000000
ABEILARD GOULART DE SOUZA FILHO,NaN,NaN,0.040146,35.795660
ABEL DO PORTO SANTOS,NaN,NaN,NaN,NaN
ABEL MOACIR BURGDURFF DE MORAES,NaN,NaN,0.298000,5960.000000
ABIDENE SALUSTIANO DA SILVA,NaN,NaN,0.107000,382.142857


In [22]:
df_party = df.groupby(['name', 'year'])['party'].max().unstack().reset_index()
df_party.rename(columns={
    2010.0: '2010-partido',
    2014.0: '2014-partido',
    2018.0: '2018-partido',
}, inplace=True)

del df_party.columns.name
df_party.set_index('name', inplace=True)
df_party.reset_index()

df_party.head()

,2010-partido,2014-partido,2018-partido
name,,,
ABDALA HABIB FRAXE JUNIOR,PTN,PTN,PODE
ABEILARD GOULART DE SOUZA FILHO,NaN,SD,PDT
ABEL DO PORTO SANTOS,PRTB,NaN,PRB
ABEL MOACIR BURGDURFF DE MORAES,NaN,PSTU,PSTU
ABIDENE SALUSTIANO DA SILVA,NaN,PSC,PSC


In [23]:
dd = pd.concat([df_evolution, df_party], axis=1, sort=False)
dd = dd[['2010-partido', '2010-2014', '2010-2014-percent', '2014-partido', '2014-2018', '2014-2018-percent']]
dd.head()

,2010-partido,2010-2014,2010-2014-percent,2014-partido,2014-2018,2014-2018-percent
name,,,,,,
ABDALA HABIB FRAXE JUNIOR,PTN,0.0,0.0,PTN,0.000000,0.000000
ABEILARD GOULART DE SOUZA FILHO,NaN,NaN,NaN,SD,0.040146,35.795660
ABEL DO PORTO SANTOS,PRTB,NaN,NaN,NaN,NaN,NaN
ABEL MOACIR BURGDURFF DE MORAES,NaN,NaN,NaN,PSTU,0.298000,5960.000000
ABIDENE SALUSTIANO DA SILVA,NaN,NaN,NaN,PSC,0.107000,382.142857


In [24]:
party_evolution_2010 = dd.groupby(['2010-partido'])['2010-2014'].sum()
party_count_2010 = dd.groupby(['2010-partido'])['2010-2014'].count()


evolution_2010_proportional = party_evolution_2010/party_count_2010 
evolution_2010_proportional_sorted = evolution_2010_proportional.sort_values(ascending=False)[0:10]
evolution_2010_proportional_sorted.to_csv('/mnt/data/top-10-partidos-que-mais-enriqueceram-2010a2014.csv')

evolution_2010_proportional_sorted

2010-partido
PR      1.059006
DEM     0.762060
PSDB    0.604462
PSDC    0.584179
PP      0.581336
PMN     0.542390
PTB     0.521047
PSC     0.489934
PV      0.478101
PDT     0.364981
Name: 2010-2014, dtype: float64

In [25]:
party_evolution_2014 = dd.groupby(['2014-partido'])['2014-2018'].sum()
party_count_2014 = dd.groupby(['2014-partido'])['2014-2018'].count()

evolution_2014_proportional = party_evolution_2014/party_count_2014
evolution_2014_proportional_sorted = evolution_2014_proportional.sort_values(ascending=False)[0:10]
evolution_2014_proportional_sorted.to_csv('/mnt/data/top-10-partidos-que-mais-enriqueceram-2014a2018.csv')

evolution_2014_proportional_sorted

2014-partido
DEM     1.563673
PSDB    0.916195
PSD     0.697548
PR      0.693890
PP      0.473686
PSB     0.434132
PMDB    0.416924
PROS    0.312837
PTN     0.267217
PDT     0.196840
Name: 2014-2018, dtype: float64

# Análise de Declaração Patrimonial por Partido

In [26]:
assets_2018 = pd.concat([df_party['2018-partido'], df_final['2018-soma-bens']], axis=1)

In [27]:
party_assets_sum = assets_2018.groupby(['2018-partido'])['2018-soma-bens'].sum()
party_person_count = assets_2018.groupby(['2018-partido'])['2018-soma-bens'].count()

party_declared_mean_asset_values = party_assets_sum / party_person_count
party_declared_mean_asset_values_ordered = party_declared_mean_asset_values.sort_values(ascending=False)[0:10]

party_declared_mean_asset_values_ordered.to_csv('/mnt/data/top-10-partidos-que-ricos-eleicoes-2018.csv')
party_declared_mean_asset_values_ordered

2018-partido
PDT     3.947403
DEM     3.600192
PSDB    3.041721
PP      2.840747
NOVO    2.630915
PR      2.363240
PSD     2.132794
PRB     1.853996
PODE    1.134835
PSB     0.964551
Name: 2018-soma-bens, dtype: float64

# Análise em cidades com menos de 100 mil habitantes

Primeiro carregamos a lista das cidades com mais de 100 mil habitantes:

In [28]:
def to_int(value):
    return int(re.sub(r'\D', '', value))

url = "https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_acima_de_cem_mil_habitantes"
biggest_cities = pd.read_html(url)[0]
biggest_cities.drop(0, inplace=True)  # remove header (interpreted as data)
biggest_cities.drop(0, axis=1, inplace=True)  # remove position column
biggest_cities.columns = ('city', 'state', 'population')  # add proper headers
biggest_cities.population = biggest_cities.population.apply(to_int)  # convert population to int
biggest_cities.city = biggest_cities.city.apply(lambda x: unidecode(x).lower())  # normalize city names
biggest_cities.head()

,city,state,population
1,sao paulo,São Paulo,12106920
2,rio de janeiro,Rio de Janeiro,6520266
3,brasilia,Distrito Federal,3039444
4,salvador,Bahia,2953986
5,fortaleza,Ceará,2627482


Agora carregamos os municípios de filiação dos candidatos:

In [29]:
cities = pd.read_csv(
    '/mnt/data/filiados-selecionados.csv.gz',
    dtype={'numero_da_inscricao': np.str}
)
cities.data_da_filiacao = pd.to_datetime(cities.data_da_filiacao, format='%Y-%m-%d', errors='coerce')
cities.sort_values('data_da_filiacao', ascending=False, inplace=True)
cities.head()

,id,codigo_do_municipio,data_da_desfiliacao,data_da_filiacao,data_da_regularizacao,data_do_cancelamento,data_do_processamento,datahora_da_extracao,motivo_do_cancelamento,nome_do_filiado,nome_do_municipio,nome_do_partido,numero_da_inscricao,secao_eleitoral,sigla_do_partido,situacao_do_registro,tipo_do_registro,uf,zona_eleitoral,search_data
3013,16574638,21490,NaN,2018-04-08,NaN,NaN,2018-04-14,2018-04-23 00:45:01+00,NaN,WASHINGTON JOAO DOS SANTOS,PUXINANÃ,PARTIDO SOCIAL LIBERAL,015169421210,55,PSL,REGULAR,OFICIAL,PB,50,'joa':5 'liberal':3 'oficial':11 'part':1 'pb'...
703,11039301,3018,NaN,2018-04-07,NaN,NaN,2018-04-14,2018-04-23 00:41:55+00,NaN,MARIA GRACILENE ABREU FERREIRA,BOA VISTA,PARTIDO DA REPÚBLICA,001655422666,394,PR,REGULAR,OFICIAL,RR,5,'abreu':6 'boa':8 'ferreir':7 'gracilen':5 'ma...
3520,12526137,93734,NaN,2018-04-07,NaN,NaN,2018-04-14,2018-04-23 00:42:38+00,NaN,MARIA TEREZINHA DO PRADO MONTEIRO,GOIÂNIA,PARTIDO RENOVADOR TRABALHISTA BRASILEIRO,020120821066,3,PRTB,REGULAR,OFICIAL,GO,1,'brasileir':4 'go':14 'goiân':10 'mar':5 'mont...
436,11856244,6050,NaN,2018-04-07,NaN,NaN,2018-04-14,2018-04-23 00:42:22+00,NaN,MANOEL DO NASCIMENTO CAMARA SOUZA,MACAPÁ,PARTIDO REPUBLICANO DA ORDEM SOCIAL,001017762542,257,PROS,REGULAR,OFICIAL,AP,10,'ap':15 'cam':9 'macap':11 'manoel':6 'nascime...
2156,19977125,97012,NaN,2018-04-07,NaN,NaN,2018-04-14,2018-04-23 00:47:10+00,NaN,EURIPEDES VIANA SANTANA,BRASÍLIA,PARTIDO TRABALHISTA CRISTÃO,008167502089,191,PTC,REGULAR,OFICIAL,DF,18,'brasíl':7 'cristã':3 'df':11 'euriped':4 'ofi...


In [30]:
cities = pd.read_csv('/mnt/data/filiados-selecionados.csv.gz')
cities.numero_da_inscricao = cities.numero_da_inscricao.astype('category')

def fstat(rows):
    return pd.Series({
            'numero_da_inscricao': rows.iloc[0]['numero_da_inscricao'].astype(np.str).zfill(12),
            'nome_do_municipio': unidecode(rows.iloc[0]['nome_do_municipio']).lower()
        })
    
cities = cities.groupby('numero_da_inscricao', as_index=False).apply(fstat)
cities.head()

,numero_da_inscricao,nome_do_municipio
0,000018152054,brasilia
1,000027090400,canoas
2,000035971317,belem
3,000042990744,fortaleza
4,000043740159,sao paulo


In [31]:
df = pd.merge(df, cities, left_on='voter_id', right_on='numero_da_inscricao')
df.head()

,id,name,voter_id,num_of_elections_run,year,state,party,position,result,sum_of_assets_declared,num_of_assets_declared,numero_da_inscricao,nome_do_municipio
0,530217,ALFREDO CARDOSO COSTA,000481911325,7,2010,PA,PT,DEPUTADO(A) ESTADUAL,SUPLENTE,0.519815,14,000481911325,belem
1,530217,ALFREDO CARDOSO COSTA,000481911325,7,2014,PA,PT,DEPUTADO(A) ESTADUAL,SUPLENTE,1.049679,12,000481911325,belem
2,530217,ALFREDO CARDOSO COSTA,000481911325,7,2018,PA,PT,DEPUTADO(A) ESTADUAL,0,0.814371,11,000481911325,belem
3,1179243,DIRCEU TEN CATEN PIES,055759191309,2,2014,PA,PT,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.136148,3,055759191309,maraba
4,1179243,DIRCEU TEN CATEN PIES,055759191309,2,2018,PA,PT,DEPUTADO(A) ESTADUAL,0,0.376456,9,055759191309,maraba


Agora excluímos os candidatos dos cidades com mais de 100 mil habitantes:

In [32]:
smaller_cities_df = df[~df.nome_do_municipio.isin(biggest_cities.city)]
smaller_cities_df.head()

,id,name,voter_id,num_of_elections_run,year,state,party,position,result,sum_of_assets_declared,num_of_assets_declared,numero_da_inscricao,nome_do_municipio
14,532967,WESCLEY SILVA AGUIAR,049509131368,5,2014,PA,PSC,DEPUTADO(A) ESTADUAL,SUPLENTE,0.260000,2,049509131368,itaituba
15,532967,WESCLEY SILVA AGUIAR,049509131368,5,2018,PA,PSC,DEPUTADO(A) ESTADUAL,0,0.307000,4,049509131368,itaituba
32,1179196,JAQUES DA SILVA NEVES,021791471333,2,2014,PA,PSC,DEPUTADO(A) ESTADUAL,ELEITO,0.414925,4,021791471333,capanema
33,1179196,JAQUES DA SILVA NEVES,021791471333,2,2018,PA,PSC,DEPUTADO(A) ESTADUAL,0,0.171500,2,021791471333,capanema
51,883572,ANTONIO JOSE SOUZA PARACAMPO,022536041333,3,2010,PA,PR,DEPUTADO(A) ESTADUAL,SUPLENTE,0.159000,4,022536041333,bujaru


# Maior evolução patrimonial em reais nas da última eleição pra essa (2014 e 2018)

In [33]:
df_final.sort_values('2014-2018', ascending=False).head(30)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent,2014-2018-percent-pretty
name,,,,,,,,
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.619703,DEPUTADO(A) ESTADUAL,153.719389,153.099686,24705.339246,24705.3%
NILSO JOSE BERLANDA,DEPUTADO(A) ESTADUAL,SUPLENTE,6.266501,DEPUTADO(A) ESTADUAL,64.809477,58.542976,934.221071,934.2%
IVONE MEZZOMO CASSOL,SENADOR(A),NÃO ELEITO,56.276546,1º SUPLENTE,84.714269,28.437724,50.532106,50.5%
DILMAR DAL BOSCO,DEPUTADO(A) ESTADUAL,ELEITO POR QP,2.394865,DEPUTADO(A) ESTADUAL,22.252168,19.857304,829.161791,829.2%
MARINALDO ROSENDO DE ALBUQUERQUE,DEPUTADO(A) FEDERAL,ELEITO POR QP,13.026175,DEPUTADO(A) FEDERAL,32.235112,19.208937,147.464145,147.5%
MIGUEL MOUBADDA HADDAD,DEPUTADO(A) FEDERAL,ELEITO POR QP,11.383318,DEPUTADO(A) FEDERAL,29.316575,17.933257,157.539799,157.5%
ALFREDO COTAIT NETO,2º SUPLENTE,NÃO ELEITO,4.746242,1º SUPLENTE,16.050417,11.304175,238.171048,238.2%
REINALDO AZAMBUJA SILVA,GOVERNADOR(A),ELEITO,27.440416,GOVERNADOR(A),38.698697,11.258282,41.028102,41.0%
ANTONIO JORGE GODINHO,DEPUTADO(A) ESTADUAL,SUPLENTE,1.649338,SENADOR(A),12.261525,10.612187,643.421067,643.4%


# Maior evolução patrimonial em valores absolutos acima da média e que tenham domicílio eleitoral em cidades com menos de 100 mil habitantes

Nesse caso, pode ser gente eleita na eleição passada ou não. Pegar os 30 com maior evolução percentual nos resultados restantes.

In [34]:
smaller_cities_final_df = df_final.reset_index()[~df_final.reset_index().name.isin(smaller_cities_df['name'])]
smaller_cities_final_df.sort_values('2014-2018', ascending=False).head(30)

,name,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent,2014-2018-percent-pretty
526,DILMAR DAL BOSCO,DEPUTADO(A) ESTADUAL,ELEITO POR QP,2.394865,DEPUTADO(A) ESTADUAL,22.252168,19.857304,829.161791,829.2%
1848,MIGUEL MOUBADDA HADDAD,DEPUTADO(A) FEDERAL,ELEITO POR QP,11.383318,DEPUTADO(A) FEDERAL,29.316575,17.933257,157.539799,157.5%
147,ALFREDO COTAIT NETO,2º SUPLENTE,NÃO ELEITO,4.746242,1º SUPLENTE,16.050417,11.304175,238.171048,238.2%
2124,REINALDO AZAMBUJA SILVA,GOVERNADOR(A),ELEITO,27.440416,GOVERNADOR(A),38.698697,11.258282,41.028102,41.0%
241,ANTONIO JORGE GODINHO,DEPUTADO(A) ESTADUAL,SUPLENTE,1.649338,SENADOR(A),12.261525,10.612187,643.421067,643.4%
640,ELI ROSA DA SILVA,DEPUTADO(A) ESTADUAL,SUPLENTE,1.649528,DEPUTADO(A) ESTADUAL,10.530000,8.880472,538.364336,538.4%
1895,NELSI COGUETTO MARIA,DEPUTADO(A) ESTADUAL,SUPLENTE,2.138991,DEPUTADO(A) FEDERAL,8.682793,6.543802,305.929367,305.9%
563,EDENILSO ROSSI ARNALDI,DEPUTADO(A) FEDERAL,0,27.077399,DEPUTADO(A) FEDERAL,33.589670,6.512271,24.050580,24.1%
1013,HELENILSON CUNHA PONTES,SENADOR(A),NÃO ELEITO,8.784208,1º SUPLENTE,15.139381,6.355173,72.347705,72.3%
102,ALCIBIO MESQUITA BIBO NUNES,DEPUTADO(A) FEDERAL,SUPLENTE,2.906126,DEPUTADO(A) FEDERAL,9.049000,6.142874,211.376723,211.4%


# Maior evolução patrimonial em porcentagem em cidades com menos de 100 mil habitantes

In [44]:
pd.options.display.float_format = '{:.5f}'.format

smaller_cities_final_df[smaller_cities_final_df['2014-soma-bens'] > 0] \
    .sort_values('2014-2018-percent', ascending=False) \
    .head(30)

,name,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent,2014-2018-percent-pretty
2298,RUBENS SANCHEZ PROENCA,DEPUTADO(A) FEDERAL,0,0.00000,DEPUTADO(A) FEDERAL,0.01200,0.01200,39999900.00000,39999900.0%
1881,NAIARA LOURENCO,DEPUTADO(A) DISTRITAL,NÃO ELEITO,0.02000,DEPUTADO(A) FEDERAL,2.04600,2.02600,10130.00000,10130.0%
992,GUSTAVO DE BIASE NUNES DA ROCHA,DEPUTADO(A) FEDERAL,SUPLENTE,0.00250,DEPUTADO(A) FEDERAL,0.17100,0.16850,6740.00000,6740.0%
318,BESALIEL DE OLIVEIRA RODRIGUES,DEPUTADO(A) ESTADUAL,NÃO ELEITO,0.01300,DEPUTADO(A) ESTADUAL,0.88000,0.86700,6669.23077,6669.2%
3,ABEL MOACIR BURGDURFF DE MORAES,1º SUPLENTE,NÃO ELEITO,0.00500,DEPUTADO(A) ESTADUAL,0.30300,0.29800,5960.00000,5960.0%
2394,SILVIO LUIZ RODRIGUES RAMALHO,DEPUTADO(A) ESTADUAL,SUPLENTE,0.00500,DEPUTADO(A) ESTADUAL,0.24400,0.23900,4780.00000,4780.0%
354,CARLOS BIANCHINI PONTES,DEPUTADO(A) DISTRITAL,SUPLENTE,0.01100,DEPUTADO(A) DISTRITAL,0.43800,0.42700,3881.81818,3881.8%
1402,JULIO CESAR SIQUEIRA,DEPUTADO(A) ESTADUAL,SUPLENTE,0.03575,DEPUTADO(A) ESTADUAL,0.97000,0.93425,2613.28671,2613.3%
2203,ROBERTO SALAME FILHO,DEPUTADO(A) FEDERAL,ELEITO POR QP,0.02443,DEPUTADO(A) FEDERAL,0.62950,0.60507,2476.66448,2476.7%
1237,JOHNSON HELEN SOARES MICHILES,DEPUTADO(A) ESTADUAL,SUPLENTE,0.01000,DEPUTADO(A) ESTADUAL,0.24000,0.23000,2300.00000,2300.0%
